In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import csv

## Part 1: Plotting the troughs of CPI, Core CPI, PCE, and Core PCE compared to the troughs during recessions. 
We will examine the following recessions: 
* Recession of 1980 - 1982 (January 1980 - July 1980 and July 1981 to November 1982)
* Early 1990s (July  1990 -  March 1991)
* Early 2000s (March 2001 - November 2001)
* Great Recession (December 2007 - June 2009)
* COVID-19 (February 2020 - April 2020)

In [3]:
# CPI historical data: https://fred.stlouisfed.org/series/CPILFESL
# Core CPI historical data: https://fred.stlouisfed.org/series/CPILFESL

# Loading the data. Converted from xlsx to csv to avoid encoding errors.
core_cpi = pd.read_csv('data/core_cpi.csv')
overall_cpi = pd.read_csv('data/overall_cpi.csv')

In [4]:
# Gathering data for each recession with data from the 4 years before and after for core CPI. 
early_80s_core = core_cpi.loc[core_cpi['observation_date'].str.startswith(('1976', '1977', '1978','1979','1980', '1981', '1982','1983', '1984', '1985', '1986'))]
early_90s_core = core_cpi.loc[core_cpi['observation_date'].str.startswith(('1986', '1987', '1988','1989','1990', '1991', '1992', '1993', '1994', '1995'))]
early_00s_core = core_cpi.loc[core_cpi['observation_date'].str.startswith(('1997', '1998', '1999', '2000','2001', '2002', '2003', '2004', '2005'))]
great_recession_core = core_cpi.loc[core_cpi['observation_date'].str.startswith(('2003', '2004', '2005', '2006','2007', '2008', '2009', '2010', '2011', '2012', '2013'))]
covid_core = core_cpi.loc[core_cpi['observation_date'].str.startswith(('2017', '2018', '2019','2020', '2021', '2022'))]

# Data was processed using Pandas and graphs were generated with Excel. Find the data and graphs at: https://drive.google.com/drive/folders/15GL9dAO53Bxe-mPjl-OXPAlRjFiy_Nn_?usp=share_link

## Part 2: Calculating the difference between average minimum inflation for the above recessions and average inflation. 

World Bank Data: https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG?locations=US

With Excel, we get the following results 
                                        
1980 - 1982 (January 1980 - July 1980 and July 1981 to November 1982):
* 1978 - 1984	
* Average Inflation: 8.059107141	
* Minimum Inflation: 3.212435233

Early 1990s (July 1990 - March 1991):		
* 1988 - 1993	
* Average Inflation: 4.086356864	
* Minimum Inflation: 2.951656966

Early 2000s (March 2001 - November 2001): 
* Time Period: 1998 - 2003	
* Average Inflation: 2.299910214	
* Minimum Inflation: 1.552279099

Great Recession (December 2007 - June 2009):		
* Time Period: 2005 - 2011	
* Average Inflation: 2.535971781
* Minimum Inflation: -0.35554627

COVID-19 (February 2020 - April 2020):		
* Time Period: 2018 - 2021	
* Average Inflation: 2.546559158	
* Minimum Inflation: 1.233584396

## Part 3: Plot the difference between wage inflation and CPI inflation. 

First, we compute average wage inflation for each year. Then we compare against the post-processed World Bank data for consumer price inflation. 

In [5]:
# Wage inflation data: https://www.atlantafed.org/chcs/wage-growth-tracker

yearly_wage_growth = {}

wage_growth = pd.read_csv('data/us_wage_growth.csv')
years =  wage_growth[['Year ', 'Unweighted Overall']]

# Helper function to return true if string can be converted into float. 
def is_float (string):
    try: 
        float(string)
        return True
    except ValueError: 
        return False

# Adds the wage inflation totals to our dictionary for each year. 
for i, elt in years.iterrows(): 
    ends_in = str(elt['Year ']).split('/')[-1]
    if (ends_in != 'nan' and is_float(elt['Unweighted Overall'])):
        ends_in = int(ends_in)
        yearly_wage_growth[ends_in] = yearly_wage_growth[ends_in] + float(elt['Unweighted Overall']) if ends_in in yearly_wage_growth else float(elt['Unweighted Overall'])

# Post-processing for average calculations. 
yearly_wage_growth[97] = yearly_wage_growth[97] / 10
yearly_wage_growth[22] = yearly_wage_growth[22] / 11

for k in yearly_wage_growth: 
    if (k != 97 and k != 22):
        yearly_wage_growth[k] = yearly_wage_growth[k] / 12

# Output as CSV for graphing. 
with open('data/wage_averages.csv', 'w') as output: 
    writer = csv.writer(output)
    for k, v in yearly_wage_growth.items(): 
        writer.writerow([k, v])

## Part 4: Performing the Granger-Causality Test to determine if prices cause wages or wages cause prices. 

In [6]:
from statsmodels.tsa.stattools import grangercausalitytests

In [16]:
wage_cpi_data = pd.read_csv('data/granger_test.csv')
grangercausalitytests(wage_cpi_data[['Average', 'Consumer Price Inflation']], maxlag = [3])


Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.9016  , p=0.4634  , df_denom=15, df_num=3
ssr based chi2 test:   chi2=3.9669  , p=0.2651  , df=3
likelihood ratio test: chi2=3.6471  , p=0.3022  , df=3
parameter F test:         F=0.9016  , p=0.4634  , df_denom=15, df_num=3


{3: ({'ssr_ftest': (0.9015578563687248, 0.4634465638095221, 15.0, 3),
   'ssr_chi2test': (3.9668545680223883, 0.26506555277925214, 3),
   'lrtest': (3.6471258356843705, 0.30217569678618267, 3),
   'params_ftest': (0.9015578563687215, 0.463446563809524, 15.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0.]])])}

In [15]:
grangercausalitytests(wage_cpi_data[['Consumer Price Inflation', 'Average']], maxlag = [3])


Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.9264  , p=0.4520  , df_denom=15, df_num=3
ssr based chi2 test:   chi2=4.0762  , p=0.2534  , df=3
likelihood ratio test: chi2=3.7395  , p=0.2910  , df=3
parameter F test:         F=0.9264  , p=0.4520  , df_denom=15, df_num=3


{3: ({'ssr_ftest': (0.9264007283463401, 0.45204731098331696, 15.0, 3),
   'ssr_chi2test': (4.076163204723897, 0.2533564578259448, 3),
   'lrtest': (3.7395414428718254, 0.2909972277271648, 3),
   'params_ftest': (0.9264007283463426, 0.45204731098331563, 15.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0.]])])}